In [ ]:
import os
os.environ['EPICS_CA_ADDR_LIST'] = 's-qat03-l.abtlus.org.br'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
from IPython.display import display
from scipy.special import erf

from bluesky import RunEngine
from bluesky.utils import Msg, make_decorator
from bluesky.plans import scan, list_scan
from bluesky.plan_stubs import open_run, close_run, one_nd_step, move_per_step
from bluesky.callbacks import LiveFitPlot, LiveFit, LivePlot
from bluesky.callbacks.best_effort import BestEffortCallback
from bluesky.suspenders import SuspendCeil

from ophyd import Device, EpicsSignal, EpicsSignalWithRBV, EpicsSignalRO, Component as Cpt
from ophyd.sim import SynGauss, SynAxis, det1, motor1, noisy_det
from ophyd.status import SubscriptionStatus

from databroker import Broker

import lmfit

In [ ]:
RE = RunEngine({})
bec = BestEffortCallback()
db = Broker.named('temp')
RE.subscribe(db.insert)
RE.subscribe(bec)

### Device Ophyd

A implementação do device pode seguir a mesma ideia dos dias anteriores. Com uma pequena mudança para o SubscriptionStatus monitorar o valor do trigger.

In [ ]:
class SampleDetector(Device):
    exposure_time = Cpt(EpicsSignalWithRBV, "ExposureTime", kind='hinted')
    trigger_signal = Cpt(EpicsSignalWithRBV, "Trigger", kind='omitted')
    data_signal = Cpt(EpicsSignalRO, "Data_RBV", kind='hinted', auto_monitor=False)
  

    def trigger(self):
        super_sts = super().trigger()
        def check_value(*, old_value, value, **kwargs):
            return (value == 0 and old_value == 1)

        sts = SubscriptionStatus(self.trigger_signal, check_value, run=False)
        self.trigger_signal.set(1).wait()

        return super_sts & sts

In [ ]:
i0 = SampleDetector('TEST:DET1:', name='i0')

### Plano de Scan em energia

O plano de scan em energia pode ser realizado com um list_scan. Basta lembrar que os scans padrões aceitam objetos __movable__, isto é, qualque device que implemente e interface de __movable__ pode ser utilizado em scans para modificar parâmetros durante um scan. Nesse exemplo vamos generalizar para múltiplos sinais de energias (são movables!!) e múltiplos sinais de tempo (são movables!!)

In [ ]:
def energy_scan(detectors, energy_list, exposure_time, energy_signals, time_signals=None):

    # Verificar se o exposure time é um float, nesse caso geramos uma lista com dimensões de energy_list
    if isinstance(exposure_time, float):
        exposure_time = [exposure_time]*len(energy_list)

    
    if time_signals is None:
        time_signals = [det.exposure_time 
                        for det in detectors 
                        if hasattr(det, 'exposure_time')]

    # Agrupamos os argumentos da lista em tuplas para associar cada energy/time signal ao seu respectivo parâmetro de configuração
    list_args = [(movable, energy_list) for movable in energy_signals]
    list_args += [(signal, exposure_time) for signal in time_signals]
    
    # Aqui temos um processo de "achatamento" da lista de tuplas, basicamente transformamos tudo em uma lista plana: [Signal, list, Signal, list, ...]
    list_args = [item 
                 for items in list_args 
                 for item in items]
    print(list_args)

    # Por fim utilizamos * para dar unpack dos argumentos posicionais da nossa lista para list_scan
    _md = {'plan_name': 'energy_scan', 'e0': 7112, 'element': 'Fe', 'edge': 'K', 'd_spacing': 0.001}
    yield from list_scan(detectors, *list_args, md=_md)

In [ ]:
i0.wait_for_connection(timeout=4)

In [ ]:
uid, = RE(energy_scan([i0], np.arange(7000, 7100, 10), [0.1, 5, 10, 0.4, 3,5,6,7,2,1], [motor1], [i0.exposure_time]))

### Metadados definidos no scan

In [ ]:
last_run = db[uid]

In [ ]:
last_run.start

### Outros exemplos: Dispositivos simulados

In [ ]:
def gaussian_integral(x, peak, sigma, center):
    return peak * sigma * np.sqrt(np.pi / 2) * (erf((x - center) / (np.sqrt(2) * sigma)) - erf((-5 - center) / (np.sqrt(2) * sigma)))

class SynKnifeDetector(SynGauss):
    
    def __init__(self, name, motor, motor_field, center, Imax, *, random_state=None, **kwargs):
        super().__init__(name, motor, motor_field, center, Imax, **kwargs)

    def _compute(self):
        m = self._motor.read()[self._motor_field]["value"]
        Imax = self.Imax.get()
        center = self.center.get()
        sigma = self.sigma.get()
        noise = self.noise.get()
        noise_multiplier = self.noise_multiplier.get()
        
        return gaussian_integral(m, Imax, sigma, center)

In [ ]:
motor1.delay = 0.15
detector = SynKnifeDetector('detector', motor1, 'motor1', center=2, Imax=9, sigma=1)

In [ ]:
fig, ax = plt.subplots()
bec.disable_plots()

    
model = lmfit.Model(gaussian_integral)

init_guess = {'peak': 5, 'sigma': 1.5, 'center': 3}


live_fit = LiveFit(model, 'detector', {'x': 'motor1'}, init_guess)
live_fit_plot = LiveFitPlot(live_fit, color='r', ax=ax, label='Fit')
live_plot = LivePlot('detector', 'motor1', marker='x', linestyle='none', ax=ax, label='Scan')

RE(
    scan([detector], motor1, -10, 10, num=40),
    [live_fit_plot, live_plot]
)

In [ ]:
live_fit.result

In [ ]:
my_dataset = db[uid].table()

In [ ]:
my_dataset.to_csv('my_data')

### Um pouco mais sobre os planos

É possível definir per_steps personalizados. Um exemplo é um per_step que toma __n__ medidas por step do scan padrão.

In [ ]:
from bluesky.plan_stubs import checkpoint, abs_set, trigger_and_read, create, save, trigger, read, wait
from functools import partial

def my_per_step(detectors, step, pos_cache, multi_triger_num=1):
    motors = step.keys()
    yield from move_per_step(step, pos_cache) # Já tem um checkpoint inserido!
    for i in range(multi_triger_num):
        yield from trigger_and_read(list(detectors) + list(motors))

In [ ]:
uid, = RE(scan([det1], motor1, -10, 10, 100, per_step=partial(my_per_step, multi_triger_num=2)))

### Aprofundando um pouco mais

In [ ]:
def simplified_trigger_and_read(devices):
    def inner():
        def read_plan():
            reading_from_devices = {}
            for device in devices:
                reading = yield from read(device)
                reading_from_devices.update(reading)
            return reading_from_devices
        for device in devices:
            yield from trigger(device, group='id_for_trigger_group')
        yield from wait(group='id_for_trigger_group')
        
        yield from create('primary')
        read_from_devices = yield from read_plan()
        yield from save()
        return read_from_devices
    return (yield from inner())

In [ ]:
def my_simple_plan(detectors, reading_plan=None):
    yield from open_run()
    if reading_plan is None:
        yield from trigger_and_read(detectors)
    else:
        yield from reading_plan(detectors)
    yield from close_run()

In [ ]:
RE(my_simple_plan([det1], simplified_trigger_and_read))

In [ ]:
RE(my_simple_plan([det1]))

In [ ]:
def my_total_custom_per_step(detectors, step, pos_cache, multi_triger_num=1):
    motors = step.keys()
    yield from move_per_step(step, pos_cache) # Já tem um checkpoint inserido!
    for i in range(multi_triger_num):
        yield from simplified_trigger_and_read(list(detectors) + list(motors))

In [ ]:
uid, = RE(scan([i0], motor1, -10, 10, 100, per_step=partial(my_total_custom_per_step, multi_triger_num=2)))

In [ ]:
RE.stop()

In [ ]:
bec.peaks